# Initial-EDA

In here I will try to determin the health of the data:

    - looking for missing entries

    - deciding how best to deal with them

    - looking for data types and formats that should be chaged

    - univariate analysis of the features

    - looking for Time Series Analysis (TSA) specific components

      - Trend
      - Seasonality
      - Cyclic
      - Irregularity

    - Identifying the Time Series Data type

      - Stationary or Non-Stationary
      - Using statistical tests to check if the data is Stationary
        1. Augmented Dickey-Fuller (ADF) Test
        2. Kwiatkowski-Plillips-Schmidt-Shin (KPSS) Test

    - Converting data to Stationary type
      - Detrending
      - Deferencing

# Imports

In [1]:
import sys
sys.path.append('../')

from src.fetch_data import DataLoader
from src.rotating_logs import get_rotating_log

In [2]:
data_path = 'data/raw/store.csv'
version = 'raw_data'
repo = '../'

store_df = DataLoader.dvc_get_data(data_path, version, repo)

DataLoaderLogger - INFO - DVC: CSV file read with path: data/raw/store.csv | version: raw_data | from: ../


In [4]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB
